In [7]:
import numpy as np
import scipy.optimize
import os
import re
import json
import matplotlib.pyplot as plt

In [80]:
tmp_dir = os.path.join(
    os.environ["USERPROFILE"], "rzgdatashare", "RZG_Data"
)
tail_name = "20181018_LD780_L4_MMF5-22-105_L11_Coherence"
var_format = "{:s}_v_x_{:d}um"
var_list = list(np.arange(4900, 5201, 10))
var_title = "LD780, L4, MMF5-22-105, L11"
var_label = "Lateral displacement (µm)"

var_folders = [
    os.path.join(tmp_dir, tail_name, var_format.format(tail_name, x))
    for x in var_list
]
for folder in var_folders:
    if not os.path.isdir(folder):
        raise FileNotFoundError(folder)

In [81]:
coh_totmax = []
coh_max = []
coh_center = []
totmax = [0, 0, 0]      # [x, y, val]
coh_maxtrace = []
for folder in var_folders:
    im = -np.load(os.path.join(folder, os.path.basename(folder) + "_imrec_min.npy"))
    
    _totmax = np.unravel_index(np.argmax(im), im.shape)
    coh_totmax.append(im[_totmax])
    if im[_totmax] > totmax[2]:
        totmax[0], totmax[1] = _totmax[0], _totmax[1]
        totmax[2] = im[_totmax]
    
    xcut, ycut = im.shape[0] // 3, im.shape[1] // 3
    im_max = im[xcut:2 * xcut, ycut:2 * ycut]
    coh_max.append(np.max(im_max))
    
    xcut, ycut = im.shape[0] // 2, im.shape[1] // 2
    im_center = im[xcut - 10:xcut + 11, ycut - 10:ycut + 11]
    coh_center.append(np.max(im_center))
    
for folder in var_folders:
    im = -np.load(os.path.join(folder, os.path.basename(folder) + "_imrec_min.npy"))
    coh_maxtrace.append(im[totmax[0], totmax[1]])

In [82]:
%matplotlib
var_list = np.array(var_list) - 5000
plt.plot(var_list, coh_totmax, label="totmax")
plt.plot(var_list, coh_max, label="max")
plt.plot(var_list, coh_center, label="center")
plt.plot(var_list, coh_maxtrace, label="maxtrace")
plt.title(var_title)
plt.xlabel(var_label)
plt.ylabel("Spatial coherence")
plt.legend()
plt.show()

Using matplotlib backend: TkAgg
